In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space'
]

In [ ]:
data_train = fetch_20newsgroups(subset='train', 
                                categories=categories, 
                                shuffle=True, random_state=42)

data_test = fetch_20newsgroups(subset='test', 
                               categories=categories, 
                               shuffle=True, random_state=42)
X_train = data_train.data
y_train = data_train.target

X_test = data_test.data
y_test = data_test.target

assert(len(X_train) == len(y_train))
assert(len(X_test) == len(y_test))

In [ ]:
data_train = fetch_20newsgroups(subset='train', 
                                categories=categories, 
                                shuffle=True, random_state=42)
n_components = 5
labels = data_train.target
true_k = np.unique(labels).shape[0]

# Convert to TF-IDF format
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', use_idf=True)
X_train = vectorizer.fit_transform(data_train.data)

# Reduce dimensions
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
# lsa = make_pipeline(svd, normalizer)

# X_train = lsa.fit_transform(X_train)

In [ ]:
len(data_train.data)

2034

In [ ]:
pd.DataFrame(data_train.target).value_counts()

2    593
1    584
0    480
3    377
dtype: int64

In [ ]:
data_test = fetch_20newsgroups(subset='test', 
                               categories=categories, 
                               shuffle=True, random_state=42)

target_names = data_train.target_names

# Split a train set and test set
y_train, y_test = data_train.target, data_test.target

print("Extracting features from the test data using the same vectorizer")
X_test = vectorizer.transform(data_test.data)
# X_test = lsa.fit_transform(X_test)

Extracting features from the test data using the same vectorizer


Convolutional Neural Networks - CNNs

In [ ]:
data_train = fetch_20newsgroups(subset='train', 
                                categories=categories, 
                                shuffle=True, random_state=42)

data_test = fetch_20newsgroups(subset='test', 
                               categories=categories, 
                               shuffle=True, random_state=42)
X_train = data_train.data
y_train = data_train.target

X_test = data_test.data
y_test = data_test.target

assert(len(X_train) == len(y_train))
assert(len(X_test) == len(y_test))

Tokenization

In [ ]:
import tensorflow_datasets as tfds

tokenizer = tfds.deprecated.text.Tokenizer()

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    X_train, target_vocab_size=2**18
)

train_inputs = [tokenizer.encode(text) for text in X_train]
test_inputs = [tokenizer.encode(text) for text in X_test]

Padding

In [ ]:
MAX_LEN = max([len(sentence) for sentence in data_train])

In [ ]:

train_inputs = tf.keras.preprocessing.sequence.pad_sequences(train_inputs,
                                                             value=0,
                                                             padding="post",
                                                             maxlen=MAX_LEN)

test_inputs = tf.keras.preprocessing.sequence.pad_sequences(test_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [ ]:
class DCNN(tf.keras.Model):
    
    def __init__(self, vocab_size, emb_dim=128, nb_filters=50, FFN_units=512, nb_classes=2,
                 dropout_rate=0.1, training=False, name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size, emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2, padding="valid", activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters, kernel_size=3, padding="valid", activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4, padding="valid", activation="relu")
        self.pool = layers.GlobalMaxPool1D() # no training variable so we can
                                             # use the same layer for each
                                             # pooling step
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1, activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes, activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [ ]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(y_train))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

Dcnn = DCNN(vocab_size=VOCAB_SIZE, emb_dim=EMB_DIM, nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS, nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "./drive/My Drive/projects/CNN_for_NLP/ckpt/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
Dcnn.fit(train_inputs,
         y_train,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Epoch 1/5
64/64 [==============================] - 13s 172ms/step - loss: 1.2595 - sparse_categorical_accuracy: 0.4371
Epoch 2/5
64/64 [==============================] - 10s 162ms/step - loss: 0.5802 - sparse_categorical_accuracy: 0.8260
Epoch 3/5
64/64 [==============================] - 10s 162ms/step - loss: 0.1294 - sparse_categorical_accuracy: 0.9661
Epoch 4/5
64/64 [==============================] - 10s 161ms/step - loss: 0.0652 - sparse_categorical_accuracy: 0.9794
Epoch 5/5
64/64 [==============================] - 10s 160ms/step - loss: 0.0564 - sparse_categorical_accuracy: 0.9823


'./drive/My Drive/projects/CNN_for_NLP/ckpt/ckpt-1'

step__5 CNN-LSTM MODEL CREATION 

In [ ]:
max_features = 20000
maxlen = 100
embedding_size = 128
# Convolution
kernel_size = 5
filters = 64
pool_size = 4
# LSTM
lstm_output_size = 70
# Training
batch_size = 30
epochs = 5

In [ ]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

NameError: ignored

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

In [ ]:
data_train = fetch_20newsgroups(subset='train', 
                                categories=categories, 
                                shuffle=True, random_state=42)

data_test = fetch_20newsgroups(subset='test', 
                               categories=categories, 
                               shuffle=True, random_state=42)
x_train = data_train.data
Y_train = data_train.target

x_test = data_test.data
Y_test = data_test.target

assert(len(X_train) == len(y_train))
assert(len(X_test) == len(y_test))

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_words = 1000
max_len = 100
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)

In [ ]:
Dcnn.fit(train_inputs,
         y_train,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()